In [ ]:
# Preprocessing the data
db_conn = DatabaseConnection()
connection = db_conn.connect_to_database()
try:
    with connection.cursor() as cursor:
        sql = "SELECT id,body,source from questions"
        cursor.execute(sql)
        result = cursor.fetchall()
    
    with connection.cursor() as cursor:
        for i in result:
            if i['source']!='app':
                a= i['body'].encode('latin').decode('utf-8')
                a = Preprocessor.punctuation_remover(a)
            else:
                decoded_data = unescape(i['body'])
                parser = MyHTMLParser()
                parser.feed(decoded_data)
                a = Preprocessor.punctuation_remover(parser.string)
            pre = Preprocessor()
            lang = LanguageSeparation()
            a = Preprocessor.remove_extra_whitespace(a)
            logging.info(pre.replace_numbers(a))

            sql = "insert into auto_categorize(question_id,body,source,language) values('"+str(i['id'])+"','"+i['body']+"','"+i['source']+"','"+lang.language_detection(i['body'])+"')"
            cursor.execute(sql)
        connection.commit()

finally:
    connection.close()

In [ ]:
# creating english word list
def clean_maya_article_en():
    with open('maya_article_en.txt', 'r') as f:
        file = f.read()
    result = file.lower()
    result = Preprocessor.punctuation_remover(result)
    result = ''.join([i for i in result if not i.isdigit()])
    result = Preprocessor.remove_extra_whitespace(result)
    with open('maya_article_en.txt', 'w') as f:
        file = f.write(result)

In [92]:
# splitting train and test data
import numpy as np
from database_connection import DatabaseConnection
db_conn = DatabaseConnection()
connection = db_conn.connect_to_database()

try:
    with connection.cursor() as cursor:
        cursor.execute("SELECT distinct question_id,body,source,tags_1,t1_id FROM merged_table where t1_id is not null and question_id<90000 and lang='banglish' and t2_id!=47 and t1_id!=4 and t1_id!=5")
        train_dataset = cursor.fetchall()
        cursor.execute("SELECT distinct question_id,body,source,tags_1,t1_id FROM merged_table where t1_id is not null and question_id>89999 and lang='banglish' and t2_id!=47 and t1_id!=5 and t1_id!=4")
        test_dataset = cursor.fetchall()
finally:
    connection.close()
    

In [ ]:
# load stopwords bangla
import json

with open('stopwords_bn.json', encoding='utf-8') as json_file:
    json_data = json.load(json_file)

In [93]:
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

train_target = np.array([])
train_target_names = []
train_data = []

for i in train_dataset:
    train_target_names.append(i['tags_1'])
    train_data.append(i['body'])
    train_target = np.append(train_target, i['t1_id'])
    
text_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultinomialNB(fit_prior=False)),
])
text_clf = text_clf.fit(train_data, train_target)

In [42]:
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(train_data)
X_train_counts.shape

(25937, 8758)

In [94]:
test_target = np.array([])
test_target_names = []
test_data = []

for i in test_dataset:
    test_target_names.append(i['tags_1'])
    test_data.append(i['body'])
    test_target = np.append(test_target, i['t1_id'])
predicted = text_clf.predict(test_data)
for i in range(len(test_data)):
    if (predicted[i] != test_target[i]):
        print(test_data[i], predicted[i], test_target_names[i])
np.mean(predicted == test_target)


smoke kori 2012 theke but regular na konodin 4 5 ta khai adda te erpor 1 week o na kheye thaki mane addicted na but eka eka  korle onek durbol lage anxiety bere jay karon ki 1.0 Psychosocial
apu thanks for ans my previous question apu ami kichu importants word er meanings clear hote parchi na jemon challenge life e challenge nite hobe but challenge word tar realistic meaning ta ki koyekta example diye bujai dile valo hoto please apu 1.0 Psychosocial
raat e ghum hoy na 1.0 Psychosocial
raat e ghum hoy na 1.0 Psychosocial
ami rate ghumate park na ki korle ghum hobe plzz apa somadan den 1.0 Psychosocial
ami jeno rate ghumate pari tai apni bolen ami akhon ki korbo 1.0 Psychosocial
hello apu ami male amar age 17 amar sokaler hight thake 5 3 and bikaler hight 5 2 ata ki r ata keno hoy 1.0 Psychosocial
amar mukhe bron ekhon ki kora jay 1.0 Psychosocial
10 dine mukher skin sundor korar kono ghoroa upay ace ki apu thakle plz janaben apu 1.0 Psychosocial
nokher fungasher owshad 3.0 Health
bron d

0.86753946861763576

In [43]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(25937, 8758)